In [1]:
%%capture
!apt install -y lynx

In [2]:
!lynx -dump https://librivox.org/dubliners-by-james-joyce/|grep mp3|awk '{print $NF}'|grep 64kb|grep 'mp3$' > dubliners1.txt
!lynx -dump https://librivox.org/dubliners-by-james-joyce-2/|grep mp3|awk '{print $NF}'|grep 64kb|grep 'mp3$' > dubliners2.txt
!wget -i dubliners1.txt
!wget -i dubliners2.txt

--2024-02-16 10:16:25--  https://www.archive.org/download/dubliners_0906_librivox/dubliners_01_joyce_64kb.mp3
Resolving www.archive.org (www.archive.org)... 207.241.224.2
Connecting to www.archive.org (www.archive.org)|207.241.224.2|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://archive.org/download/dubliners_0906_librivox/dubliners_01_joyce_64kb.mp3 [following]
--2024-02-16 10:16:25--  https://archive.org/download/dubliners_0906_librivox/dubliners_01_joyce_64kb.mp3
Resolving archive.org (archive.org)... 207.241.224.2
Connecting to archive.org (archive.org)|207.241.224.2|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ia902800.us.archive.org/3/items/dubliners_0906_librivox/dubliners_01_joyce_64kb.mp3 [following]
--2024-02-16 10:16:26--  https://ia902800.us.archive.org/3/items/dubliners_0906_librivox/dubliners_01_joyce_64kb.mp3
Resolving ia902800.us.archive.org (ia902800.us.archive.org)... 207.241

In [3]:
!for i in *.mp3.1;do mv $i $(basename $i .mp3.1)_2.mp3;done

Here starts the actual ASR stuff.

In [4]:
!pip install transformers

In [5]:
EN_MODEL = "jonatasgrosman/wav2vec2-large-xlsr-53-english"
PHONE_MODEL = "vitouphy/wav2vec2-xls-r-300m-timit-phoneme"

In [6]:
from transformers import pipeline

2024-02-16 10:29:48.564055: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-16 10:29:48.564164: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-16 10:29:48.853604: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [7]:
pipe_en = pipeline(model=EN_MODEL, device=0)
pipe_phone = pipeline(model=PHONE_MODEL, device=1)

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Some weights of the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-english were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-english and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
Y

vocab.json:   0%|          | 0.00/300 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/486 [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/309 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


preprocessor_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]

In [8]:
from pathlib import Path
import json
cur = Path(".")
for file in cur.glob("*.mp3"):
    en_out = pipe_en(str(file), chunk_length_s=10, return_timestamps="word")
    phone_out = pipe_phone(str(file), chunk_length_s=10, return_timestamps="word")
    stem = file.stem
    with open(f"{stem}_en.json", "w") as enfile:
        json.dump(en_out, enfile)
    with open(f"{stem}_phone.json", "w") as phonefile:
        json.dump(phone_out, phonefile)

/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentiall

In [9]:
!rm *.mp3